In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [259]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [302]:
path = "boneage_preds_0.3"
with open("../../../../" + path, 'rb') as f:
    data = pickle.load(f)

In [303]:
preds_adv_on_1 = data['preds_adv_on_1']
preds_adv_on_2 = data['preds_adv_on_2']
preds_vic_on_1 = data['preds_vic_on_1']
preds_vic_on_2 = data['preds_vic_on_2']

In [304]:
preds_adv_on_1.preds_property_1.shape, preds_adv_on_2.preds_property_1.shape

((50, 762), (50, 645))

In [301]:
def KL(a, b):
    a_, b_ = 1 - a, 1 - b
    first = a * (np.log(a) - np.log(b))
    second = a_ * (np.log(a_) - np.log(b_))
    together = first + second
    return np.mean(together, 1) # Technique 2
    # return np.mean(together)  # Technique 1

In [305]:
def main_stuff(ka, kb, kc1, kc2):
    ka_ = sigmoid(ka)
    kb_ = sigmoid(kb)
    kc1_ = sigmoid(kc1)
    kc2_ = sigmoid(kc2)
    
    KL_vals_1_a = np.array([KL(ka_, x) for x in kc1_])
    KL_vals_1_b = np.array([KL(kb_, x) for x in kc1_])
    KL_vals_2_a = np.array([KL(ka_, x) for x in kc2_])
    KL_vals_2_b = np.array([KL(kb_, x) for x in kc2_])

    # Technique 1: Aggregate KL value across all data and models, and then compare
    # preds_first = np.mean(KL_vals_1_a < KL_vals_1_b)
    # preds_second = np.mean(KL_vals_2_b < KL_vals_2_a)
    
    # Technique 2: Aggregate KL value across all data per adv model pair (m0[i], m1[i]), then vote
    # Could technically consider all nC2 pairs, but can try that later
    # This approach seems to work better in my experiments on BoneAge
    preds_first = np.mean(np.mean(KL_vals_1_a < KL_vals_1_b, 1) >= 0.5)
    preds_second = np.mean(np.mean(KL_vals_2_b < KL_vals_2_a, 1) >= 0.5)
    
    acc = (preds_first + preds_second)/2
    return acc

In [306]:
main_stuff(preds_adv_on_1.preds_property_1,
           preds_adv_on_1.preds_property_2,
           preds_vic_on_1.preds_property_1,
           preds_vic_on_1.preds_property_2)

0.7749999999999999

In [281]:
# KL_vals_1_a = np.array([KL(x, ka_) for x in kc1_])
# KL_vals_1_b = np.array([KL(x, kb_) for x in kc1_])

# KL_vals_2_a = np.array([KL(x, ka_) for x in kc2_])
# KL_vals_2_b = np.array([KL(x, kb_) for x in kc2_])